# Q5 — Implement & Test PyTorch-Based Classifier
Includes short answer placeholders (random init, tqdm, torch.no_grad, resetting metrics), transforms, val_loader, plotting loss, and extracting preds/labels.

In [ ]:
# Short answers (fill these when submitting)
print('Q5 Theory placeholders:') 
print('- Why random init: ensures symmetry breaking between neurons so learning can occur.')
print('- tqdm: progress bar utility for loops.')
print('- Reset metrics each epoch: to avoid accumulating across epochs and get per-epoch metrics.')
print('- torch.no_grad(): avoids tracking gradients during eval to save memory/time.')
print('- Evaluation metrics: accuracy, precision, recall, F1, confusion matrix.')

# Implement pipeline
import torch, torch.nn as nn, torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

tf_train = transforms.Compose([transforms.Resize((64,64)), transforms.RandomHorizontalFlip(), transforms.ToTensor()])
tf_val = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])

train_ds = datasets.ImageFolder('images_dataSAT', transform=tf_train)
val_ds = datasets.ImageFolder('images_dataSAT', transform=tf_val)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=4, shuffle=False)

class Net(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3,16,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16,32,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((1,1))
        )
        self.fc = nn.Linear(32, num_classes)
    def forward(self,x):
        x = self.net(x).view(x.size(0), -1)
        return self.fc(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(num_classes=len(train_ds.classes)).to(device)
opt = optim.Adam(model.parameters(), lr=1e-3)
crit = nn.CrossEntropyLoss()

train_losses, val_losses = [], []
for epoch in range(3):
    model.train()
    running = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        out = model(xb)
        loss = crit(out, yb)
        loss.backward()
        opt.step()
        running += loss.item()*xb.size(0)
    train_losses.append(running/len(train_loader.dataset))
    # validation
    model.eval()
    vr = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            loss = crit(out, yb)
            vr += loss.item()*xb.size(0)
    val_losses.append(vr/len(val_loader.dataset))
    print(f'Epoch {epoch+1}: train_loss={train_losses[-1]:.4f}, val_loss={val_losses[-1]:.4f}')

# plot loss
plt.plot(train_losses, label='train_loss'); plt.plot(val_losses, label='val_loss'); plt.legend(); plt.show()

# extract predictions & labels
all_preds, all_labels = [], []
model.eval()
with torch.no_grad():
    for xb, yb in val_loader:
        xb = xb.to(device)
        logits = model(xb)
        preds = logits.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds.tolist())
        all_labels.extend(yb.numpy().tolist())

print('Confusion matrix:\n', confusion_matrix(all_labels, all_preds))
print(classification_report(all_labels, all_preds, zero_division=0))